In [2]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:80% !important;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-header{margin-top:150px;}
.cm-s-ipython span.cm-comment{font-size:12pt; font-style:normal; color:orange;}
.ansi-yellow-bg{color:black;font-size:15pt;}

</style>
<style>
span.toc-item-num{display:none;}
</style>
"""))

전국 평균 평당 분양가격(2013년 9월부터 2015년 8월까지).csv (지역명, 평당분양가격, 연도,월)
- 지역명필드는 라벨인코딩하여 추가
- 평당 분양가격n : 평당분양가격의 정규화 스케일 조정하여 추가
- 평당 분양가격s : 평당분양가격의 표준화 스케일 조정하여 추가
- 연도n : 연도의 정규화 스케일 조정하여 추가
- 연도s : 연도의 표준화 스케일 조정하여 추가
- 월 n : 월의 정규화 스케일 조정하여 추가
- 월 s : 월의 표준화 스케일 조정하여 추가

In [59]:
import pandas as pd
df = pd.read_csv("../../Downloads/shareData/전국 평균 평당 분양가격(2013년 9월부터 2015년 8월까지).csv",encoding="cp949")
df = df.melt(id_vars="지역",var_name="연도월", value_name="평당분양가격")
df["연도"] = df["연도월"].apply(lambda x : int(x.split("년")[0]))
df["월"] = df["연도월"].apply(lambda x : int(x.split("년")[-1][:-1]))
df.drop('연도월',axis= 1, inplace=True)
df = df.rename(columns={'지역':'지역명'})
df.head()


,지역명,평당분양가격,연도,월
0,서울,18189,2013,12
1,부산,8111,2013,12
2,대구,8080,2013,12
3,인천,10204,2013,12
4,광주,6098,2013,12


In [60]:
import numpy as np
from tensorflow.keras.utils import to_categorical # 분류분석시 원핫인코딩
from tensorflow.keras.models import Sequential #모델생성
from tensorflow.keras.layers import Dense # 모델에 레이어 추가시 
from tensorflow.keras.layers import Input
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

In [61]:
# 지역명필드는 라벨인코딩하여 추가
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df["지역명2"] = le.fit_transform(df["지역명"])
df.head()

,지역명,평당분양가격,연도,월,지역명2
0,서울,18189,2013,12,8
1,부산,8111,2013,12,7
2,대구,8080,2013,12,5
3,인천,10204,2013,12,11
4,광주,6098,2013,12,4


In [62]:
# 원핫인코딩한 데이터

m2_data = np.array(df["평당분양가격"]).reshape(-1,1)

#평당 분양가격n : 평당분양가격의 정규화 스케일 조정하여 추가
scaler_m2 = MinMaxScaler()
scaler_m2.fit(m2_data)
df["평당 분양가격n"] = scaler_m2.transform(m2_data)


#평당 분양가격s : 평당분양가격의 표준화 스케일 조정하여 추가
from sklearn.preprocessing import StandardScaler
scaler_m2 = StandardScaler()
scaler_m2.fit(m2_data)
df["평당분양가격s"] = scaler_m2.transform(m2_data)


In [63]:
# 원핫인코딩한 데이터

y_data = np.array(df['연도']).reshape(-1,1)# df['연도'].values.reshape(-1,1)
#연도n : 연도의 정규화 스케일 조정하여 추가
scaler_y = MinMaxScaler()
scaler_y.fit(y_data)
df['연도s'] = scaler_y.transform(y_data)


#연도s : 연도의 표준화 스케일 조정하여 추가
from sklearn.preprocessing import StandardScaler
scaler_y = StandardScaler()
scaler_y.fit(y_data)
df["연도n"] = scaler_y.transform(y_data)

In [64]:

m_data = np.array(df["월"]).reshape(-1,1)
#월 n : 월의 정규화 스케일 조정하여 추가
scaler_m = MinMaxScaler()
scaler_m.fit(m_data)
df["월n"] = scaler_m.transform(m_data)



#월 s : 월의 표준화 스케일 조정하여 추가
from sklearn.preprocessing import StandardScaler
scaler_m = StandardScaler()
scaler_m.fit(m_data)
df["월s"] = scaler_m.transform(m_data)



In [65]:
df

,지역명,평당분양가격,연도,월,지역명2,평당 분양가격n,평당분양가격s,연도s,연도n,월n,월s
0,서울,18189,2013,12,8,0.835270,3.297226,0.0,-2.366432,1.000000,1.767475
1,부산,8111,2013,12,7,0.166124,-0.113291,0.0,-2.366432,1.000000,1.767475
2,대구,8080,2013,12,5,0.164066,-0.123782,0.0,-2.366432,1.000000,1.767475
3,인천,10204,2013,12,11,0.305093,0.595005,0.0,-2.366432,1.000000,1.767475
4,광주,6098,2013,12,4,0.032468,-0.794515,0.0,-2.366432,1.000000,1.767475
...,...,...,...,...,...,...,...,...,...,...,...
352,전북,6580,2015,8,13,0.064471,-0.631400,1.0,1.183216,0.636364,0.589158
353,전남,6289,2015,8,12,0.045150,-0.729878,1.0,1.183216,0.636364,0.589158
354,경북,7037,2015,8,3,0.094814,-0.476746,1.0,1.183216,0.636364,0.589158
355,경남,7665,2015,8,2,0.136512,-0.264223,1.0,1.183216,0.636364,0.589158
